In [2]:
import pandas as pd

In [9]:
ny_data = pd.read_csv("./New_York_State_Statewide_COVID-19_Testing.csv")
county_codes = pd.read_csv("./NY_Municipalities_and_County_FIPS_codes.csv")
county_data = pd.read_csv("https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/counties.csv")

In [10]:
#fixing name error
county_codes.replace("St Lawrence","St. Lawrence",inplace=True)

In [11]:
#joining infection data with county code data
ny_data_fips = ny_data.set_index("County").join(county_codes.set_index("County Name")).astype({'County FIPS': 'int32'})

In [12]:
#setting columns we will use for SES and infection related variables
relevant_ses_columns = ["Median_Household_Income_2018","transit_scores - population weighted averages aggregated from town/city level to county"]
relevant_infection_columns = ['Test Date','New Positives','Cumulative Number of Positives','Total Number of Tests Performed','Cumulative Number of Tests Performed',]

In [13]:
#data for testing is divided into municipalities for each county - we are aggregating it into county level data
grouped_county_inf = ny_data_fips.groupby(['County FIPS','Test Date'])[relevant_infection_columns].sum().reset_index()

In [14]:
#joining infection and SES data
ses_and_infection = county_data[county_data['State']=='NY'].set_index('FIPS').join(grouped_county_inf.set_index('County FIPS'))

In [15]:
#selecting relevant columns
ses_and_infection = ses_and_infection[relevant_ses_columns+relevant_infection_columns].drop(36000)

In [16]:
#resetting index to create county FIPS column
ses_and_infection = ses_and_infection.reset_index().rename(columns={'index':'FIPS'})

In [17]:
#creating binary column that denotes whether test date is before or after stay at home order: 1 = After
ses_and_infection['Test Date'] = pd.to_datetime(ses_and_infection['Test Date'])
ses_and_infection['After Order'] = ses_and_infection['Test Date'].apply(lambda x: int(x > pd.to_datetime('03/22/2020')))

In [18]:
#saving to df
ses_and_infection.to_csv("./ny_county_data/ny_county_data.csv")